## BUENOS AMIGOS


```
$git clone https://github.com/cd-x/plant-detect
```
![Home Page](https://github.com/cd-x/plant-disease/blob/master/screenshot.png)

# READ - 1

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import random

Mounting drive so that we can store dataset

In [ ]:
from google.colab import files,drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Here is a fun talk about how i got the dataset

**Attempt-1:-** using tensorflow dataset which you can find over [here](https://arxiv.org/abs/1511.08060) 


```
import tensorflow_datasets as tfds
data =  tfds.image_classification.plant_village.PlantVillage
```
I didn't know much about it but still managed to get the data and splitted it since tfds has its own builtin methods it provides API for all the operations like splitting it according to your need. Although because of lack of tutorials i left it.

**Attempt-2**:- with data collected from git repo of @spMohanty link below,


```
!git clone https://github.com/spMohanty/PlantVillage-Dataset

TRAIN_DIR= os.path.join('/content/PlantVillage-Dataset/raw/color')
print(len(os.listdir(TRAIN_DIR)))
```
trained it with the RMSProp optimizer with LEARNING_RATE = 0.0001
```
from tensorflow.keras.optimizers import RMSprop
```
splitting data with validation_split = 0.3 ,
**VALIDATION ACCURACY:** with Augementation: 80.38 (apx.) and without augmentation : 86.48(apx.). 

training with augmntation was a lot slower and yes u're right it sucked so bad and guess what testing accuracy on unseen data was 6% and 10% . I know it was awful, it was suffering from overfitting. 

**ATTEMPT-3:-** with the dataset given below which gave me good accuracy, yes not Validation_accuracy.

In [ ]:
!git clone https://gitlab.com/huix/leaf-disease-plant-village.git

fatal: destination path 'leaf-disease-plant-village' already exists and is not an empty directory.


creating path to our dataset

In [ ]:
data_dir=os.path.join(os.getcwd(),'/content/leaf-disease-plant-village/plantvillage_deeplearning_paper_dataset/')
image_dir=os.path.join(data_dir,'color')

The sizes that we'll be using throughout the project

In [ ]:
IMAGE_SIZE=(224,224)
BATCH_SIZE=64

**Creating Data-generators**

ImageDatagenerator gives a lot fuctionality like splitting data at runtime shuffling data before passing it to the neural net, rescaling, **adding augmentation** etc.

datagenerator creates kind of a pipeline between data directory and model.

**In need we can add the augmentation in following way**
```
 augment = True #@param {type:"boolean"}
 if augment:
   training_datagen = ImageDataGenerator(
       rescale = 1./255,
       rotation_range=40,
       horizontal_flip=True,
       width_shift_range=0.2, 
       height_shift_range=0.2,
       shear_range=0.2, 
       zoom_range=0.2,
       validation_split=0.2,
       fill_mode='nearest' )
else:
  training_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
```
**USES:**


*   rescale:

    Rescale 1./255 is to transform every pixel value from range [0,255] -> [0,1]. And the benefits are: Treat all images in the same manner: some images are high pixel range, some are low pixel range. ... Scaling every images to the same range [0,1] will make images contributes more evenly to the total loss.
*   rotation:

  rotate=40 rotates the image by 40 degree



*   horizontal_flip:

  flip the image horizontally

*   vertical_flip:

    flips the image vertically

*   width_shift:

  shifts the image width wise
*   hight_shift:

  shifts the image height wise


*   shear_range:

  changes shear_value of each image


*   zoom_range:

  zoom in each image and create new one to pass into NN.

*   fill_mode:

  replace noisy and absent pixel value with the nearest(if given) pixel value.(smoothing)





In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.2)

training_generator = datagen.flow_from_directory(image_dir,
                                                 shuffle=False,
                                                 seed=42,
                                                 color_mode="rgb",
                                                 class_mode="categorical",
                                                 subset="training",
                                                 target_size=IMAGE_SIZE,
                                                 batch_size=BATCH_SIZE)

validation_generator = datagen.flow_from_directory(
    image_dir, 
    shuffle=False, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    subset="validation",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

Found 43456 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.


The base model here is MobileNetV2 which is pretrained model defined in keras applications. instead of creating your own model you can just pass this as your first layer remember to make base_model.trainable =False other wise it will also train this base model and it will consume a lot of training time. 

**ATTEMP-1:-** using my own model

```
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # 38 output neuron. It will contain a value from 0-1 where values will represent confidence for each class
    tf.keras.layers.Dense(38, activation='softmax')
])
```
compiled it using RMSProp having learning rate =1e-4 for 15 epochs
```
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['acc'])
```
![trained graph](https://github.com/cd-x/plant-detect/graph.png)

it gave accuracy of 0.8989 which was nothing but hokum it failed on unseen images

**ATTEMPT-2:-** using tensorflow_hub it worked and gave accuracy of 0.9333 but i had to let it go because of deployment issue and a suugestion if possible avoid using tf hub if you want to deploy your model or if u want to ru on a local machine it has integration issue. But it is best for kaggle competitions because of good accuracy and less training time.

Here is how you can do it
```
model = tf.keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", 
                 output_shape=[1280],
                 trainable=False),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(training_generator.num_classes, activation='softmax')
])
```
**ATTEMPT-3:-** using keras applications the one i used below

In [ ]:
IMG_SHAPE=(224,224,3)
base_model = tf.keras.applications.MobileNetV2(include_top=False,input_shape=IMG_SHAPE,weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


model architecture

In [ ]:
base_model.trainable=False
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

Laying it out

In [ ]:
model = tf.keras.Sequential([
                             base_model,
                             tf.keras.layers.Dropout(0.4),
                             tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dense(512,activation='relu'),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(training_generator.num_classes,activation='softmax')
])

compiling the model with optimzer given as **Adam**

In [ ]:
LEARNING_RATE = 0.001

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss = 'categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
EPOCHS = 10

history = model.fit(
    training_generator,
    steps_per_epoch = training_generator.samples//training_generator.batch_size,
    epochs=EPOCHS,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples//validation_generator.batch_size)

Epoch 1/10
679/679 [==============================] - 1919s 3s/step - loss: 1.7853 - accuracy: 0.5507 - val_loss: 0.8113 - val_accuracy: 0.7635
Epoch 2/10
679/679 [==============================] - 1897s 3s/step - loss: 0.6797 - accuracy: 0.7887 - val_loss: 0.4328 - val_accuracy: 0.8531
Epoch 3/10
679/679 [==============================] - 1876s 3s/step - loss: 0.4513 - accuracy: 0.8519 - val_loss: 0.3341 - val_accuracy: 0.8841
Epoch 4/10
679/679 [==============================] - 1886s 3s/step - loss: 0.3722 - accuracy: 0.8747 - val_loss: 0.2519 - val_accuracy: 0.9181
Epoch 5/10
679/679 [==============================] - 1887s 3s/step - loss: 0.3081 - accuracy: 0.8954 - val_loss: 0.2436 - val_accuracy: 0.9141
Epoch 6/10
679/679 [==============================] - 1915s 3s/step - loss: 0.2824 - accuracy: 0.9035 - val_loss: 0.2135 - val_accuracy: 0.9273
Epoch 7/10
679/679 [==============================] - 1893s 3s/step - loss: 0.2351 - accuracy: 0.9208 - val_loss: 0.1823 - val_accuracy:

save the model in .h5 extension below is just to show you that running it will give you variable index directory one i have talked on READ 2

In [ ]:
model.save('acc9275own.h5py')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: acc9275own.h5py/assets


Its the way you can zip the 'variable-index directory from mounted drive' so that you can download

In [ ]:
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

if __name__ == '__main__':
    zipf = zipfile.ZipFile('Python.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir('/content/acc9275own.h5py', zipf)
    zipf.close()

**Beow TWO cells are also explained in READ-2 (modelTest2.ipynb) refer that**

In [ ]:
import json

with open('categories.json', 'r') as f:
    cat_to_name = json.load(f)
    classes = list(cat_to_name.values())
    
print (classes)

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Sp

In [ ]:
# Import OpenCV
import cv2

# Utility
import itertools
import random
from collections import Counter
from glob import iglob

def load_image(filename):
    #img = cv2.imread(filename)
    img = cv2.imread(os.path.join(image_dir, filename)) #<-- use in case of test through existing validation dataset
    img = cv2.resize(img, (IMAGE_SIZE[0], IMAGE_SIZE[1]) )
    img = img /255
    
    return img


def predict(image):
    probabilities = model.predict(np.asarray([img]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}

**Testing our model with validation directory by taking 5 random samples**

to see exaples of testing you can refer current.ipynb in **more_info directory** 

In [ ]:
for idx, filename in enumerate(random.sample(validation_generator.filenames, 5)):
    print("SOURCE: class: %s, file: %s" % (os.path.split(filename)[0], filename))
    
    img = load_image(filename)
    prediction = predict(img)
    print("PREDICTED: class: %s, confidence: %f" % (list(prediction.keys())[0], list(prediction.values())[0]))
    plt.imshow(img)
    plt.figure(idx)    
    plt.show()

# Reloading model

In [ ]:
from tensorflow.keras.models import load_model
import os

In [ ]:
model = load_model('acc9275own.h5')